In [15]:
import numpy as np
import pandas as pd

import sys
sys.path.append('..')
from lib import *

In [27]:
k = 9
proteome = 'Malaria'
model = 'nskewfcov'
reference = 'Human'
q = naminoacids

In [28]:
entropy = pd.read_csv('data/{proteome}_{model}_k{k}_entropy.csv'.format(proteome=proteome, model=model, k=k),
                      header=None, index_col=0)
entropy_reference = pd.read_csv('data/{proteome}_{model}_k{k}_entropy.csv'.format(proteome=reference, model=model, k=k),
                      header=None, index_col=0)

params = np.load('data/{proteome}_{model}_k{k}_params.npz'.format(proteome=proteome, model=model, k=k))
params_reference = np.load('data/{proteome}_{model}_k{k}_params.npz'.format(proteome=reference, model=model, k=k))

In [29]:
def make_energy(params):
    if params.files == ['f']:
        raise NotImplementedError('independent model dkl not implemented')
    elif params.files == ['h', 'J']:
        model = 'ncov'
        h = params['h']
        J = params['J']

        @njit
        def energy(x):
            return energy_ncov(x, h, J)
    elif params.files == ['h', 'J', 'J2']:
        model = 'nskew'
        h = params['h']
        J = params['J']
        J2 = params['J2']

        @njit
        def energy(x):
            return energy_nskew(x, h, J, J2)
    elif params.files == ['h', 'J', 'J2', 'hi', 'Jij']:
        model = 'nskewfcov'
        h = params['h']
        hi = params['hi']
        J = params['J']
        J2 = params['J2']
        Jij = params['Jij']

        @njit
        def energy(x):
            return energy_nskewfcov(x, h, J, J2, hi, Jij)
    return energy

In [30]:
energy = make_energy(params)
energy_reference = make_energy(params_reference)

In [31]:
matrix = load_matrix('data/{proteome}_{model}_k{k}_matrix.csv.gz'.format(proteome=proteome, model=model, k=k))
energies = np.array([energy(x) for x in matrix])
energies_reference = np.array([energy_reference(x) for x in matrix])

In [41]:
DKL = float(entropy.loc['F']) - np.mean(energies) + np.mean(energies_reference) - float(entropy_reference.loc['F'])

In [42]:
DKL

1.9752893560514195

In [36]:
entropy.loc['S']

1    24.507198
Name: S, dtype: float64

In [39]:
k = 9
np.log(q)*k-entropy.loc['S']

1    2.454392
Name: S, dtype: float64